In [2]:
from pyspark.sql import SparkSession, dataframe
from pyspark.sql.functions import when, col, sum, count, isnan, round
from pyspark.sql.functions import regexp_replace, concat_ws, sha2, rtrim, substring
from pyspark.sql.functions import unix_timestamp, from_unixtime, to_date
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql import HiveContext
from pyspark.sql.functions import year, month, dayofmonth, quarter
from pyspark.sql.types import DecimalType
from pyspark.sql.functions import trim, regexp_replace, when, col

import os
import re

from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import when

spark = SparkSession.builder.master("local[*]")\
    .enableHiveSupport()\
    .getOrCreate()

RuntimeError: Java gateway process exited before sending its port number

In [80]:
def salvar_df(df, file):
    output = "input/projeto-hive/gold/" + file
    erase = "hdfs dfs -rm " + output + "/*"
    rename = "hdfs dfs -get /datalake/gold/"+file+"/part-* input/projeto-hive/gold/"+file+".csv"
    
    print(rename)    
    
    df.coalesce(1).write\
        .format("csv")\
        .option("header", True)\
        .option("delimiter", ";")\
        .mode("overwrite")\
        .save("/datalake/gold/"+file+"/")
    
    os.system(erase)
    os.system(rename)

In [81]:
# df_clientes = spark.sql("select * from desafio_curso.clientes")
# df_divisao = spark.sql("select * from desafio_curso.divisao")
# df_regiao = spark.sql("select * from desafio_curso.regiao")
# df_vendas = spark.sql("select * from desafio_curso.vendas")

# df_endereco = spark.read.table("desafio_curso.endereco")
# rdd = df_endereco.rdd.zipWithIndex().filter(lambda x: x[1] > 0).map(lambda x: x[0])
# df_endereco = rdd.toDF(df_endereco.schema)

df_clientes = spark.sql("select * from desafio_curso.clientes")
df_divisao = spark.sql("select * from desafio_curso.divisao")
df_regiao = spark.sql("select * from desafio_curso.regiao")
df_vendas = spark.sql("select * from desafio_curso.vendas")

# Carregar tabela endereco e remover a primeira linha que copiou o cabeçalho
df_endereco = spark.read.table("desafio_curso.endereco")
rdd = df_endereco.rdd.zipWithIndex().filter(lambda x: x[1] > 0).map(lambda x: x[0])
df_endereco = rdd.toDF(df_endereco.schema)

# Carregar tabela clientes e remover a primeira linha que copiou o cabeçalho
df_clientes = spark.read.table("desafio_curso.clientes")
rdd = df_clientes.rdd.zipWithIndex().filter(lambda x: x[1] > 0).map(lambda x: x[0])
df_clientes = rdd.toDF(df_clientes.schema)

# Carregar tabela divisao e remover a primeira linha que copiou o cabeçalho
df_divisao = spark.read.table("desafio_curso.divisao")
rdd = df_divisao.rdd.zipWithIndex().filter(lambda x: x[1] > 0).map(lambda x: x[0])
df_divisao = rdd.toDF(df_divisao.schema)

# Carregar tabela regiao e remover a primeira linha que copiou o cabeçalho
df_regiao = spark.read.table("desafio_curso.regiao")
rdd = df_regiao.rdd.zipWithIndex().filter(lambda x: x[1] > 0).map(lambda x: x[0])
df_regiao = rdd.toDF(df_regiao.schema)


df_endereco.limit(3).show()
df_clientes.limit(3).show()
df_divisao.show()
df_regiao.show()


+--------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+-----+------------+
|address_number|                city|country|  customer_address_1|  customer_address_2|  customer_address_3|  customer_address_4|state|    zip_code|
+--------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+-----+------------+
|      10000000|               Akron|     US|         PO Box 6258|                 ...|                 ...|                 ...|   OH|       44312|
|      10000453|                 ...|     UK|                 ...|                 ...|                 ...|                 ...|     |            |
|      10000455|    Huntington Beach|     US|   7392 Count Circle|                 ...|                 ...|                 ...|   CA|       92647|
+--------------+--------------------+-------+--------------------+--------------------+-------------------

In [82]:
#Fazer um tabelão

df_stage_clientes_localidade = df_clientes.join(df_endereco, "address_number", "inner")\
                                .join(df_divisao, "division", "inner")\
                                .join(df_regiao, "region_code", "inner")\
                                .join(df_vendas, "customer_key", "inner")

df_stage_clientes_localidade.show()

+------------+-----------+--------+--------------+--------------------+-------------+-----------+-------------+----------------+------------+------------------+-----------+--------+-------+-------------------+--------------------+--------------------+--------------------+-----+--------+-------------+-----------+--------------------+----------+---------------+------------+--------------+----------+-----------+--------------------+-----------+----------+------------+----------------------+------------+--------------------------------+-----------------+-------------------+-----------+--------------+---------+---+
|customer_key|region_code|division|address_number|business_family_name|business_unit|   customer|customer_type|line_of_business|       phone|regional_sales_mgr|search_type|    city|country| customer_address_1|  customer_address_2|  customer_address_3|  customer_address_4|state|zip_code|division_name|region_name|actual_delivery_date|  date_key|discount_amount|invoice_date|invoice_

In [83]:
#Adicionar colunas Ano, Mês, Dia, Trimestre tomando como base a coluna invoice_date

df_stage_clientes_localidade = df_stage_clientes_localidade.withColumn('invoice_date', to_date(col('invoice_date'), 'dd/MM/yyyy'))

df_stage = df_stage_clientes_localidade \
    .withColumn('Ano', year('invoice_date')) \
    .withColumn('Mes', month('invoice_date')) \
    .withColumn('Dia', dayofmonth('invoice_date')) \
    .withColumn('Trimestre', quarter('invoice_date'))

df_stage.show()
                        

+------------+-----------+--------+--------------+--------------------+-------------+-----------+-------------+----------------+------------+------------------+-----------+--------+-------+-------------------+--------------------+--------------------+--------------------+-----+--------+-------------+-----------+--------------------+----------+---------------+------------+--------------+----------+-----------+--------------------+-----------+----------+------------+----------------------+------------+--------------------------------+-----------------+-------------------+-----------+--------------+---------+---+----+---+---+---------+
|customer_key|region_code|division|address_number|business_family_name|business_unit|   customer|customer_type|line_of_business|       phone|regional_sales_mgr|search_type|    city|country| customer_address_1|  customer_address_2|  customer_address_3|  customer_address_4|state|zip_code|division_name|region_name|actual_delivery_date|  date_key|discount_amoun

In [84]:
#Campos decimais ou inteiros nulos ou vazios, deversão ser preenchidos por 0.

cols_to_check = ['item_number', 'discount_amount', 'list_price', 'sales_amount', 'sales_amount_based_on_list_price', 'sales_cost_amount', 'sales_margin_amount', 'sales_price', 'line_number', 'sales_quantity']

for col_name in cols_to_check:
     df_stage = df_stage.withColumn(col_name, when(col(col_name) == '', 0).otherwise(col(col_name)))


In [85]:
#Filtro para verificar se determinado cliente que tinha valores vazio foram substituidos por zero
df_stage.filter((col("customer_key") == "10016588") & (col("invoice_number") == "110753")).show(truncate=False)

+------------+-----------+--------+--------------+--------------------+-------------+----------------+-------------+----------------+------------+------------------+-----------+-------+-------+---------------------+----------------------------------------+----------------------------------------+----------------------------------------+-----+----------+-------------+-----------+--------------------+----------+---------------+------------+--------------+----------+-----------+------------------------------+-----------+----------+------------+----------------------+------------+--------------------------------+-----------------+-------------------+-----------+--------------+---------+---+----+---+---+---------+
|customer_key|region_code|division|address_number|business_family_name|business_unit|customer        |customer_type|line_of_business|phone       |regional_sales_mgr|search_type|city   |country|customer_address_1   |customer_address_2                      |customer_address_3       

In [86]:
#Campos strings vazios deverão ser preenchidos com 'Não informado'
all_columns = df_stage.columns

for column in all_columns:
    df_stage = df_stage.withColumn(column, 
                                   when(trim(regexp_replace(col(column), '\n', '')) == "", "Nao informado")
                                   .otherwise(col(column)))

#Filtro para verificar se determinado cliente que tinha strings vaziasforam substituídos com 'Não informado'
df_stage.filter((col("customer_key") == "10016588") & (col("invoice_number") == "110753")).show(truncate=False)


+------------+-----------+--------+--------------+--------------------+-------------+----------------+-------------+----------------+------------+------------------+-----------+-------+-------+---------------------+------------------+------------------+------------------+-----+----------+-------------+-----------+--------------------+----------+---------------+------------+--------------+-------------+-----------+------------------------------+-----------+----------+------------+----------------------+------------+--------------------------------+-----------------+-------------------+-----------+--------------+---------+---+----+---+---+---------+
|customer_key|region_code|division|address_number|business_family_name|business_unit|customer        |customer_type|line_of_business|phone       |regional_sales_mgr|search_type|city   |country|customer_address_1   |customer_address_2|customer_address_3|customer_address_4|state|zip_code  |division_name|region_name|actual_delivery_date|date_key

In [87]:
df_stage.filter((col("customer_key") == "10016588") & (col("invoice_number") == "110753")).show()


+------------+-----------+--------+--------------+--------------------+-------------+----------------+-------------+----------------+------------+------------------+-----------+-------+-------+--------------------+------------------+------------------+------------------+-----+----------+-------------+-----------+--------------------+----------+---------------+------------+--------------+-------------+-----------+--------------------+-----------+----------+------------+----------------------+------------+--------------------------------+-----------------+-------------------+-----------+--------------+---------+---+----+---+---+---------+
|customer_key|region_code|division|address_number|business_family_name|business_unit|        customer|customer_type|line_of_business|       phone|regional_sales_mgr|search_type|   city|country|  customer_address_1|customer_address_2|customer_address_3|customer_address_4|state|  zip_code|division_name|region_name|actual_delivery_date|  date_key|discount_

In [88]:
# # Conta o número de valores vazios em todas as colunas
# empty_counts = [df_stage.select(when(col(c) == "", 1).otherwise(0)).agg({"*": "sum"}).collect()[0][0] for c in df_stage.columns]
# for i, c in enumerate(df_stage.columns):
#     print(f"{c}: {empty_counts[i]}")

In [89]:
# # Verificar se há valores nulos em colunas numéricas
# from pyspark.sql.functions import isnan, col
# from pyspark.sql.types import DoubleType, IntegerType, LongType, FloatType

# numeric_cols = [c.name for c in df_stage.schema if c.dataType in [DoubleType(), IntegerType(), LongType(), FloatType()]]

# null_counts = [df_stage.select(isnan(col(c)).alias(c)).agg({"*": "sum"}).collect()[0][0] for c in numeric_cols]
# for i, c in enumerate(numeric_cols):
#     print(f"{c}: {null_counts[i]}")

In [90]:
#Adicionar chaves estrangeiras
df_stage = df_stage.withColumn('PK_TEMPO', sha2(concat_ws("",df_stage.invoice_date, df_stage.Ano,df_stage.Mes,df_stage.Dia,df_stage.Trimestre), 256))

df_stage = df_stage.withColumn('PK_CLIENTES', sha2(concat_ws("",df_stage.customer_key,df_stage.customer,df_stage.business_family_name,df_stage.business_unit,df_stage.customer_type,df_stage.division,df_stage.line_of_business,df_stage.phone,df_stage.region_code,df_stage.regional_sales_mgr,df_stage.search_type), 256))

df_stage = df_stage.withColumn('PK_LOCALIDADE', sha2(concat_ws("",df_stage.address_number,df_stage.city,df_stage.country,df_stage.state,df_stage.zip_code,df_stage.division,df_stage.division_name,df_stage.region_code,df_stage.region_name,df_stage.customer_address_1,df_stage.customer_address_2,df_stage.customer_address_3,df_stage.customer_address_4), 256))

In [91]:
df_stage.show(5, truncate=False)

+------------+-----------+--------+--------------+--------------------+-------------+-----------+-------------+----------------+------------+------------------+-----------+--------+-------+-------------------+------------------+------------------+------------------+-----+--------+-------------+-----------+--------------------+----------+---------------+------------+--------------+----------+-----------+-------------------------+-----------+----------+------------+----------------------+------------+--------------------------------+-----------------+-------------------+-----------+--------------+---------+---+----+---+---+---------+----------------------------------------------------------------+----------------------------------------------------------------+----------------------------------------------------------------+
|customer_key|region_code|division|address_number|business_family_name|business_unit|customer   |customer_type|line_of_business|phone       |regional_sales_mgr|searc

In [92]:
df_stage.createOrReplaceTempView("stage")

In [93]:
#Gerar dimensões e fato a partir do tabelão  
df_tempo = spark.sql("SELECT DISTINCT PK_TEMPO, invoice_date, Ano, Mes, Dia, Trimestre FROM stage")

In [94]:
#Exportar dataframes como tabelas csv
salvar_df(df_tempo, 'dim_tempo')

hdfs dfs -get /datalake/gold/dim_tempo/part-* input/projeto-hive/gold/dim_tempo.csv


In [95]:
df_clientes = spark.sql("SELECT DISTINCT PK_CLIENTES, customer_key, customer, business_family_name, business_unit, customer_type, division, line_of_business, phone, region_code, regional_sales_mgr, search_type FROM stage")

In [96]:
salvar_df(df_clientes, 'dim_clientes')

hdfs dfs -get /datalake/gold/dim_clientes/part-* input/projeto-hive/gold/dim_clientes.csv


In [97]:
df_localidade = spark.sql("SELECT DISTINCT PK_LOCALIDADE, address_number, city, country, state, zip_code, division, division_name, region_code, region_name, customer_address_1, customer_address_2, customer_address_3, customer_address_4 FROM stage")

In [98]:
salvar_df(df_localidade, 'dim_localidade')

hdfs dfs -get /datalake/gold/dim_localidade/part-* input/projeto-hive/gold/dim_localidade.csv


In [99]:
ft_vendas = spark.sql("SELECT PK_CLIENTES, PK_TEMPO, PK_LOCALIDADE, actual_delivery_date, date_key, discount_amount, invoice_date, invoice_number, item_class, item_number, item, line_number, list_price, order_number, promised_delivery_date, sales_amount, sales_amount_based_on_list_price, sales_cost_amount, sales_margin_amount, sales_price, sales_quantity, sales_rep, u_m  FROM stage")  

In [100]:
salvar_df(ft_vendas, 'ft_vendas')

hdfs dfs -get /datalake/gold/ft_vendas/part-* input/projeto-hive/gold/ft_vendas.csv
